# Graded exercise: Constraining Models

## Problem description

Based on the ball-and-stick ipython notebook, write a new optimisation according to the following criteria:
- This optimisation should keep the same parameters as we already have, but add an extra parameter 'gleak_hh'. This represent the size of the leak current in the 'hh' mechanism of NEURON. The bounds of this parameter should be [1e-6, 1e-5]
- To constrain this parameter better, we will include a trace with negative current injection to the protocol. The current injected should be -0.1 nA, leave the length and delay of the current injection the same as for the amplitudes.
- For the trace generated by this current injection, use the efeature 'steady_state_voltage_stimend'.
- Run the optimisation with an offspring_size of 30, and 10 generations.

Run this optimisation, and send us the parameters of the best individual.

## Solution

In [10]:
%matplotlib inline
import matplotlib.pyplot as plt
import bluepyopt as bpop
import bluepyopt.ephys as ephys

# ...

# An example answer of the expected output
best_ind_dict_ex = {'gl_hh': 0.0, 'gnabar_hh': 0.0, 'gkbar_hh': 0.0}
answer = '%f,%f,%f' % (best_ind_dict_ex['gnabar_hh'], best_ind_dict_ex['gkbar_hh'], best_ind_dict_ex['gl_hh'])
print('Answer: %s' % answer)

Answer: 0.000000,0.000000,0.000000


# Submit answer

First install the python package to contact the grader

In [11]:
!pip -q install --upgrade --pre -i https://bbpteam.epfl.ch/repository/devpi/simple/ single-cell-mooc-client
import single_cell_mooc_client as sc_mc
submission_widget = sc_mc.Submission()

In the widget above copy-paste the output of the print(answer) statement.

(So for the example answer above "0.000000,0.000000,0.000000" WITHOUT the double quotes)